# Data Science 100 Final Project Report

In [1]:
#Loading the relevant R package:
library(tidyverse)
library(tidymodels)
library(repr)
library(themis)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [2]:
# load the dataframes
players<- read_csv("https://raw.githubusercontent.com/ninzyt/dsci100-project-group37/refs/heads/main/players.csv")
sessions<- read_csv("https://raw.githubusercontent.com/ninzyt/dsci100-project-group37/refs/heads/main/sessions.csv")
head(players)
head(sessions)

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17


hashedEmail,start_time,end_time,original_start_time,original_end_time
<chr>,<chr>,<chr>,<dbl>,<dbl>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,30/06/2024 18:12,30/06/2024 18:24,1.71977e+12,1.71977e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,17/06/2024 23:33,17/06/2024 23:46,1.71867e+12,1.71867e+12
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,25/07/2024 17:34,25/07/2024 17:57,1.72193e+12,1.72193e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,25/07/2024 03:22,25/07/2024 03:58,1.72188e+12,1.72188e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,25/05/2024 16:01,25/05/2024 16:12,1.71665e+12,1.71665e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,23/06/2024 15:08,23/06/2024 17:10,1.71916e+12,1.71916e+12


#### Summary table of the data:
| variable | type | meaning | # of missing observations | summary statistics (if applicable) | notes|
|--|--|--|--|--|--|
|`hashedEmail`|character|The hashed email address of the player for the play session (see `players` dataset summary)|0|N/A|This could be considered a factor type (despite being read in as character), since there is a limited number of values for this variable (i.e. the various hashed emails of  players from the `players` dataset). Accordingly, since this corresponds the `hashedEmail` variable in the `players` dataset, this  can be used to link information about each player to their sessions. |
|`start_time`|character|The start time and date of the play session (using 24 hour time), in the format of: "dd/mm/yyyy hour:min"|0|N/A|This variable was read as a character type, however, during wrangling it can be converted to a date type (specifically dttm), which could be beneficial in analysis/visualization due to the inherent "order" of date data.|
|`end_time`|character|The end time and date of the play session (using 24 hour time), in the format of: "dd/mm/yyyy hour:min"|2|N/A|"|
|`original_start_time`|double|The start time and date of the play session in Unix time, which is the number of non-leap seconds since midnight (UTC) of January 1, 1970|0|Max = 1.72733e+12, Min = 1.7124e+12, Mean = 1.719201e+12, Median = 1.7192e+12, Standard Deviation = 3557491589 |This represents the same information as the `start_time` variable, but using a different data type|
|`original_end_time`|double|The end time and date of the play session in Unix time, which is the number of non-leap seconds since midnight (UTC) of January 1, 1970|2|Max = 1.72734e+12, Min = 1.7124e+12, Mean = 1.719196e+12, Median = 1.71918e+12, Standard Deviation = 3552813134|This represents the same information as the `end_time` variable, but using a different data type|


**ISSUES:** 
- There are few visible issues, other than the 2 observations with missing data for both the `end_time` and `original_end_time` variable, which will need to be dealt with during later wrangling/processing.
- This should not have the issues associated with self-reported participant data, since the data is recorded by the server.
- However, since the `start_time` and `original_start_time` variables (same applies to the end time variables) contain the same information, only one is needed for each analysis, depending on needs. For example, visualizations may prefer the use of the `start_time` variable, but any modeling, etc. may benefit from the numerical format of `original_start_time`.


### Method and Results

In [6]:
# 1. Cleaning and wrangling players.csv 
players_cleaned <- players|>
                   filter(!is.na(Age))|>
                   mutate(subscribe = as.factor(subscribe))

head(players_cleaned)

experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<fct>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17


In [7]:
# 2. Summary statistics (mean values for each numerical variables)
players_average <- players_cleaned |>
                   summarize(average_played_hours = mean(played_hours), average_age = mean(Age))
players_average

average_played_hours,average_age
<dbl>,<dbl>
5.904639,20.52062


In [10]:
# 3. splitting dataframe into training, testing datasets
set.seed(1)
players_split <- initial_split(players_cleaned, prop = 0.75, strata = subscribe)

players_training <- training(players_split)
players_testing <- testing(players_split)

In [ ]:
players_recipe<- recipe(subscribe~Age+played_hours, data=players_training)|>
step_scale(all_predictors())|>
step_center(all_predictors()) |>
step_upsample(subscribe, over_ratio = 1)

In [13]:
players_spec_tune<- nearest_neighbor(weight_func="rectangular", neighbors=tune())|>
set_engine("kknn")|>
set_mode("classification")

k_vals<- tibble(neighbors=seq(from=1, to=100, by=1))
players_vfold<- vfold_cv(players_training, v=5, strata=subscribe)

player_workflow_tune<- workflow()|>
add_recipe(players_recipe)|>
add_model(players_spec_tune)|>
tune_grid(resamples=players_vfold, grid=k_vals)|>
collect_metrics()

In [12]:
accuracy<- workflow|>
filter(.metric=="accuracy")
accuracy

best_k_val<- filter(accuracy, mean==max(mean)) |>
filter(neighbors = min(neighbors))|>
pull(neighbors)

best_k_val

neighbors,.metric,.estimator,mean,n,std_err,.config
<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
1,accuracy,binary,0.4349589,5,0.05284339,Preprocessor1_Model001
2,accuracy,binary,0.4418555,5,0.05694405,Preprocessor1_Model002
3,accuracy,binary,0.3937931,5,0.05326889,Preprocessor1_Model003
4,accuracy,binary,0.3799836,5,0.05009221,Preprocessor1_Model004
5,accuracy,binary,0.4480296,5,0.04656150,Preprocessor1_Model005
6,accuracy,binary,0.4546962,5,0.04884314,Preprocessor1_Model006
7,accuracy,binary,0.5034975,5,0.05272756,Preprocessor1_Model007
8,accuracy,binary,0.5034975,5,0.05272756,Preprocessor1_Model008
9,accuracy,binary,0.4554351,5,0.04196710,Preprocessor1_Model009


ERROR: [1m[33mError[39m in `filter()`:[22m
[1m[22m[33m![39m We detected a named input.
[36mℹ[39m This usually means that you've used `=` instead of `==`.
[36mℹ[39m Did you mean `neighbors == min(neighbors)`?


In [ ]:
ggplot_accuracy<- ggplot(accuracy, aes(x=neighbors, y=mean))+
geom_point()+
geom_line()
ggplot_accuracy

In [ ]:
test_try_spec<- nearest_neighbor(weight_func = "rectangular", neighbors=19)|>
set_engine("kknn")|>
set_mode("classification")

test_fit<- workflow() |>
  add_recipe(recipe) |>
  add_model(test_try_spec) |>
  fit(data = players_training)

test_predictions <- predict(test_fit, players_testing) |>
  bind_cols(players_testing)

test_predictions |>
  metrics(truth = subscribe, estimate = .pred_class) |>
  filter(.metric == "accuracy")

test_predictions |>
    precision(truth = subscribe, estimate = .pred_class, event_level="first")

test_predictions |>
    recall(truth = subscribe, estimate = .pred_class, event_level="first")
test_predictions

confusion <- test_predictions |>
             conf_mat(truth = subscribe, estimate = .pred_class)
confusion

In [ ]:
# 4. Exploratory visualizations
# a) finding relationships for the Age predictor:
options(repr.plot.width = 8, repr.plot.height = 8)
Age_boxplot <- ggplot(players_training, aes(x = subscribe, y = Age, fill = subscribe)) +
               geom_boxplot() +
               labs(x= "Subscription Status", y= "Age of Player", fill = "Subscription\nStatus", title = "Age vs. Subscription Status")+
               theme(text = element_text(size = 15))
Age_boxplot

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8)
Age_histogram <- ggplot(players_training, aes(x = Age, fill = subscribe)) +
                 geom_histogram() +
                 facet_grid(cols = vars(subscribe))+
                 labs(x = "Age Distribution", y = "Subscription Count", fill = "Subscription\nStatus", title = "Age Distribution by Subscription Status")+
                 theme(text = element_text(size = 15))
Age_histogram

In [ ]:
# b) finding relationships for the played_hours predictor:
options(repr.plot.width = 10, repr.plot.height = 8)
played_hours_boxplot <- ggplot(players_training, aes(x = subscribe, y = played_hours, fill = subscribe)) +
                        geom_boxplot() +
                        scale_y_log10(labels=label_comma())+
                        labs(x = "Subscription Status", y = "Game Played Hours", fill = "Subscription\nStatus", title = "played hours vs. Subscription Status")+
                        theme(text = element_text(size = 15))
played_hours_boxplot